In [13]:
from operator import itemgetter
import os

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()  # carica le variabili d'ambiente dal file .env


True

# Indovinelli

In [14]:
indovinello_1 = "Qual è quell'animale che al mattino cammina a 4 zampe, a mezzogiorno con 2, alla sera con 3?"
indovinello_2 = "Cos'è quella cosa che tocca solo una persona ma ne unisce ben due?"
indovinello_3 = "Cos’è che ti tiene in vita e si vede solo d’inverno?"

In [15]:
# LLMs
mini_llm = ChatOpenAI(model="gpt-4o-mini",
                      api_key=os.getenv("OPENAI_API_KEY"),
                      temperature=0.3)

big_llm  = ChatOpenAI(model="gpt-5",
                      api_key=os.getenv("OPENAI_API_KEY"))


In [16]:
# === Prompt 1: il modello piccolo prova a indovinare ===
# Chiediamo un output JSON strutturato per evitare ambiguità.

riddle_solver_prompt = ChatPromptTemplate.from_template(
    """Sei un risolutore di indovinelli.
Indovinello: {riddle}

Fornisci la risposta indicando:
- "ipotesi": la risposta all'indovinello in UNA o poche parole.
- "spiegazione": una breve spiegazione (max 5 parole).

Rispondi SOLO seguendo queste istruzioni"""
)

riddle_solver_chain = riddle_solver_prompt | mini_llm | StrOutputParser()


In [17]:
riddle_solver_chain.invoke({"riddle": indovinello_2})

'- "ipotesi": anello\n- "spiegazione": unisce due persone in matrimonio.'

In [18]:
# === Prompt 2: il modello grande vede solo l'output del primo ===
# Obiettivo: ricostruire l’indovinello originale avendo soltanto ipotesi+spiegazione.
reconstruction_prompt = ChatPromptTemplate.from_template(
    """Hai a disposizione soltanto questa traccia, prodotta da un modello più piccolo:

{mini_output}

Prova a ricostruire l'indovinello ORIGINALE.
Se necessario, formula il miglior indovinello plausibile coerente con la traccia."""
)


In [19]:
# Catena composita
complete_chain = {
    "mini_output": riddle_solver_chain  # passa direttamente l'output JSON del primo LLM
} | reconstruction_prompt | big_llm | StrOutputParser()

In [27]:
complete_chain.invoke({"riddle": indovinello_2})

'Senza inizio né fine, brillo ma non sono stella. Al dito prometto “per sempre” e unisco due cuori. Chi sono?'

# Sintassi LangChain
Elementi:
- complete_chain
    - *riddle_solver_chain*:
        - prompt
        - mini_llm
        - StrOutputParser
    - reconstruction_prompt
    - big_llm
    - StrOutputParser

In [21]:

riddle_solver_chain = riddle_solver_prompt | mini_llm | StrOutputParser()
complete_chain = riddle_solver_chain | reconstruction_prompt | big_llm | StrOutputParser()

In [22]:
complete_chain = (riddle_solver_prompt
                | mini_llm 
                | StrOutputParser()
                | reconstruction_prompt
                | big_llm
                | StrOutputParser())

In [23]:
complete_chain = (riddle_solver_prompt
                 .pipe(mini_llm)
                 .pipe(StrOutputParser())
                 .pipe(reconstruction_prompt)
                 .pipe(big_llm)
                 .pipe(StrOutputParser()))

In [ ]:
riddle_solver_chain = riddle_solver_prompt | mini_llm | StrOutputParser()

complete_chain = (
    {"mini_output": riddle_solver_chain}  # dict => RunnableMap/Parallel implicito
    | reconstruction_prompt
    | big_llm
    | StrOutputParser()
)
complete_chain.invoke({"riddle": indovinello_3})

In [ ]:
mini_result = riddle_solver_chain.invoke({"riddle": indovinello_3})
final_result = (
    reconstruction_prompt
    | big_llm
    | StrOutputParser()
).invoke({"mini_output": mini_result})

In [ ]:
# rinomina l'output in {"mini_output": ...}
as_mini_output = RunnableLambda(lambda x: {"mini_output": x})

complete_chain = (
    riddle_solver_chain
    | as_mini_output
    | reconstruction_prompt
    | big_llm
    | StrOutputParser()
)

complete_chain.invoke({"riddle": indovinello_3})

# Content Creator

In [1]:
# ============================================
# Chain a singolo input: THEME
# Accresce il contesto e lancia in parallelo:
#  - image prompt -> image (Runway)
#  - post
# ============================================

import os
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from runwayml import RunwayML, TaskFailedError


## Configs

In [2]:

# -----------------------------
# Config (solo theme in input)
# -----------------------------
REFERENCE_IMAGE_URL = "https://www.videocitta.com/wp-content/uploads/2024/05/Andrea-Moccia-scaled.jpg"
PERSON_TAG = "Andrea"
RUNWAY_MODEL = "gen4_image"
RUNWAY_RATIO = "1920:1080"

# ENV richieste
if not os.getenv("openai_api_key"):
    raise RuntimeError("Imposta OPENAI_API_KEY.")
if not os.getenv("runway_api_key"):
    raise RuntimeError("Imposta RUNWAY_API_KEY.")

## Modelli

In [ ]:
# -----------------------------
# Modelli
# -----------------------------
increasing_llm = ChatOpenAI(model="gpt-4o",
                            api_key=os.getenv("openai_api_key"),
                            temperature=0.7)

post_llm       = ChatOpenAI(model="gpt-4o",
                            api_key=os.getenv("openai_api_key"),
                            temperature=0.9)

# # Ollama locale per image prompt
# image_prompt_llm = ChatOllama(
#     model="llama-vincent",
#     temperature=0.7,
#     num_predict=1024,
#     base_url="http://localhost:11434",
#     request_timeout=30,
# )

# GPT-3.5-turbo fine-tuned
image_prompt_llm = ChatOpenAI(model="ft:gpt-3.5-turbo-1106:personal:photorealism:BqgCFSkM",
                            api_key=os.getenv("openai_api_key"),
                            temperature=0.7)

In [33]:
image_prompt_llm.invoke("Crea un prompt per generare un'immagine di un gatto spaziale")

AIMessage(content="Create an image of a spacecat, floating in zero gravity, with stars and nebulae in the background, wearing a futuristic astronaut suit, and a small jetpack on its back. The cat's fur is a mix of dark purple and deep blue, with glowing blue stripes running along its body, and bright yellow eyes reflecting the stars. Its ears are pointed, and it sports a thin, glowing ring around its neck, similar to a high-tech collar. The cat is reaching out towards a floating, holographic fish, with its tail swishing back and forth, creating small ripples in the surrounding stardust.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 25, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'ft:gpt-3.5-turbo-1106:personal:photorealism

## Prompts

In [ ]:
# Prompt per aumentare il contesto a partire dal tema
increasing_prompt = ChatPromptTemplate.from_template(
    "Dato il tema «{theme}», elenca (bullet sintetici) le caratteristiche del tono "
    "per un articolo di divulgazione scientifica destinato a un pubblico generalista. "
    "Enfatizza gli aspetti più ingaggianti."
)

# Prompt per generare il post
post_prompt = ChatPromptTemplate.from_template(
    "Contesto ampliato (tono e spunti):\n***\n{context}\n***\n"
    "Scrivi un post social molto coinvolgente (~120 parole) coerente con il tema. Call to action finale"
    "Chiudi con 2–3 hashtag pertinenti."
)

# Prompt per generare il prompt dell'immagine
image_prompt_requester = ChatPromptTemplate.from_template(
    "Forniscimi un prompt per modificare la persona nell'immagine allegata nello stile «{theme}»."
)


## Catena semplice

In [35]:
# 1) Increasing: theme -> context (string)
context_chain = (
    RunnableLambda(lambda theme: {"theme": theme}) # passa avanti il tema
    | increasing_prompt     # genera il prompt per il contesto  (quello con cui abbiamo fin-tunato il modello)
    | increasing_llm        # genera il prompt
    | StrOutputParser()     # estrae il testo
)

# 2) Post: usa solo il context
post_chain = (
    RunnableLambda(lambda d: {"context": d["context"]})  # utile per riportare il contesto in output
    | post_prompt
    | post_llm
    | StrOutputParser()
)

# 3) Image prompt: usa theme + context
image_prompt_chain = (
    RunnableLambda(lambda d: {"theme": d["theme"]})
    | image_prompt_requester
    | image_prompt_llm
    | StrOutputParser()
)


## Runway

In [36]:
# 4) Runway: genera immagine a partire dall'image_prompt
def _to_image(inputs: dict) -> dict:
    client = RunwayML(api_key=os.getenv("runway_api_key"))
    image_prompt = inputs["image_prompt"]
    try:
        task = client.text_to_image.create(
            model=RUNWAY_MODEL,
            ratio=RUNWAY_RATIO,
            prompt_text=image_prompt,
            reference_images=[{"uri": REFERENCE_IMAGE_URL,
                               "tag": PERSON_TAG
                               }],
        ).wait_for_task_output()
        url = task.output[0] if getattr(task, "output", None) else None
        return {**inputs, 
                "image_prompt": image_prompt,
                "image_url": url}
    except TaskFailedError as e:
        return {**inputs, "image_url": None, "image_error": f"{getattr(e, 'task_details', str(e))}"}

runway_node = RunnableLambda(_to_image)
runway_chain = (image_prompt_chain 
                | RunnableLambda(lambda p: {"image_prompt": p}) 
                | runway_node)


## Orchestrazione

In [37]:
# -----------------------------
# Orchestrazione
# -----------------------------
# Step A: preparazione condivisa — da theme → {theme, context}
prepare = RunnableParallel(
    theme=RunnablePassthrough(),            # passa il theme così com'è
    context=context_chain                   # elabora il context dal theme
)

# Step B: rami paralleli finali
# - ramo post: prende il context e scrive il post
# - ramo immagine: genera image_prompt (theme+context) e poi crea l'immagine
final = RunnableParallel(
    context=itemgetter("context"),          # utile riportare il contesto in output
    post=(post_chain),
    image=(runway_chain),
)

# Catena completa: theme -> prepare -> parallelo post+immagine
chain = prepare | final



## Avvio

In [42]:

theme = "L'avvento delle Intelligenze Artificiali. Cyber-futurismo, Tech, Cyber-punk"
result = chain.invoke(theme)

print("\n=== CONTEXT (accresciuto) ===\n", result["context"])
print("\n=== POST ===\n", result["post"])
print("\n=== IMAGE PROMPT ===\n", result["image"].get("image_prompt"))
print("\n=== IMAGE URL ===\n", result["image"].get("image_url"))
if result["image"].get("image_error"):
    print("\n[Runway ERROR]\n", result["image"]["image_error"])


=== CONTEXT (accresciuto) ===
 - **Equilibrato**: Mantenere un tono che bilanci entusiasmo e cautela riguardo all'avvento delle intelligenze artificiali.
- **Accessibile**: Utilizzare un linguaggio semplice e chiaro per spiegare concetti complessi, evitando il gergo tecnico.
- **Curiosità**: Stimolare l'interesse e la curiosità dei lettori verso il futuro tecnologico e le sue potenzialità.
- **Entusiasmo**: Riflettere l'energia e l'eccitazione per le innovazioni nel campo dell'IA e le loro possibili applicazioni.
- **Realismo**: Evidenziare anche le preoccupazioni e i rischi associati all'IA, senza cadere nel sensazionalismo.
- **Immaginativo**: Evocare immagini futuristiche e scenari cyber-punk per coinvolgere la fantasia dei lettori.
- **Riflessivo**: Invitare il pubblico a riflettere sulle implicazioni etiche e sociali delle tecnologie emergenti.
- **Ottimista**: Sottolineare le opportunità positive che l'IA potrebbe offrire alla società, promuovendo un senso di speranza.
- **Inter

In [43]:
print(chain)

first={
  theme: RunnablePassthrough(),
  context: RunnableLambda(lambda theme: {'theme': theme})
           | ChatPromptTemplate(input_variables=['theme'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['theme'], input_types={}, partial_variables={}, template='Dato il tema «{theme}», elenca (bullet sintetici) le caratteristiche del tono per un articolo di divulgazione scientifica destinato a un pubblico generalista. Enfatizza i sentimenti legati al tema.'), additional_kwargs={})])
           | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000026E92365F70>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026E923647D0>, root_client=<openai.OpenAI object at 0x0000026E92367200>, root_async_client=<openai.AsyncOpenAI object at 0x0000026E92367860>, model_name='gpt-4o', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('*********